In [1]:
import pandas as pd

In [ ]:
""""

Vehículos homologados antes de ~2014 COLUMNA AM.
Vehículos homologados 2014 en adelante COLUMNA  AH.
Vehículos 100 % eléctricos o de hidrógeno = 0"""

"""
Si la columna “Año modelo” ≥ 2015 → usa columna AH (WLTP)
Si la columna “Año modelo” ≤ 2014 → usa columna AM (NEDC o FTP-75)
Si “Tecnología” = BEV (Battery Electric Vehicle) o FCEV → emisiones = 0

"""

'"\n\nVehículos homologados antes de ~2014 COLUMNA AM.\nVehículos homologados 2014 en adelante COLUMNA  AH.\nVehículos 100 % eléctricos o de hidrógeno = 0'

# Homologación de vehiculos

### llenar valores faltantes CO2
